# Gridworld random policy evaluation (Example 4.1)

In [1]:
from typing import Dict, List, Optional, Set, Tuple

import abc
import enum
import numpy as np

In [2]:
class Action(enum.Enum):
    LEFT: int = 0
    RIGHT: int = 1
    UP: int = 2
    DOWN: int = 3


State = Tuple[int, int]


class GridWorld:

    _n: int
    _state: State

    _MOVE: Dict[Action, State] = {
        Action.LEFT: (0, -1),
        Action.RIGHT: (0, 1),
        Action.UP: (-1, 0),
        Action.DOWN: (1, 0),
    }

    def __init__(self, n: int, state: Optional[State] = None) -> None:
        self._n = n
    
    def transition(self, state: State, action: Action, next_state: State) -> float:
        """Return the probability of goig to `next_state` from `state` by doing `action`.
        Here, it is deterministic, we just have to check if when applied `action`,
        we go to the `next_state`.
        """
        return float(self._move(state, action) == next_state)
    
    def get_states(self) -> List[State]:
        return [ 
            (x, y)
            for y in range(self._n)
            for x in range(self._n)
        ]

    def get_non_terminal_states(self) -> List[State]:
        return [ 
            state
            for state in self.get_states()
            if state not in self.get_terminal_states()
        ]
    
    def get_terminal_states(self) -> List[State]:
        return [(0, 0), (self._n - 1, self._n - 1)]
    
    def get_reachable_states(self, state: State) -> Set[State]:
        return {self._move(state, action) for action in Action}

    def _move(self, state: State, action: Action) -> State:
        return self._bound_state(tuple(map(sum, zip(state, self._MOVE[action]))))

    def _bound_state(self, state: State) -> State:
        x, y = state
        return self._bound_coordinate(x), self._bound_coordinate(y)

    def _bound_coordinate(self, coordinate: int) -> int:
        return min(max(coordinate, 0), self._n - 1)

In [3]:
class Policy(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def probability(self, action: Action, state: State) -> float:
        """Return the probability of choosing `action` when in `state`."""
        pass


class RandomPolicy(Policy):
    def probability(self, action: Action, state: State) -> float:
        """Equal probability to choose every action."""
        return 1 / len(Action)

# Value estimation

In [4]:
n = 4
env = GridWorld(n)
gamma = 1
theta = 0.0001
policy = RandomPolicy()
states = env.get_states()
non_terminal_states = env.get_non_terminal_states()
V = {state: 0.0 for state in states}
delta = float("inf")

while delta > theta:
    delta = 0
    for state in non_terminal_states:
        v = V[state]
        V[state] = sum(
            policy.probability(action, state) * sum(
                env.transition(state, action, next_state) * (-1 + gamma * V[next_state])
                for next_state in env.get_reachable_states(state)
            )
            for action in Action
        )
        delta = max(delta, abs(v - V[state]))

In [5]:
V_array = np.zeros((n, n))
for (row, col), value in V.items():
    V_array[row, col] = value
print(np.round(V_array, 1))

[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
